In [1]:
#Importing the necessary libraries for the script
import geemap
import ee
from pathlib import Path
import geopandas as gpd
from functions import *

In [2]:
#Authenticating the Google Earth Engine API
ee.Authenticate()
ee.Initialize()
geemap.ee_initialize()

In [3]:
#Setting the Out Directory if not exists creating it
out_dir = Path('../Data')
out_dir.mkdir(exist_ok=True)

#Shapefile directory of the area of interest
shp_dir = Path('../Shapefile/BegnasArea.shp')

In [7]:
studyarea = get_eesupported_roi(shp_dir)

#Earth Engine Collection
dataset_landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2022-04-01', '2022-07-01').filter(ee.Filter.lt('CLOUD_COVER', 2))
dataset_landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate('2022-04-01', '2022-07-01').filter(ee.Filter.lt('CLOUD_COVER', 2))


In [8]:
# Function to apply scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Process and save Landsat 9 data
dataset_landsat9 = dataset_landsat9.map(apply_scale_factors)
landsat_9 = dataset_landsat9.median().clip(studyarea)
save_data(out_dir, landsat_9, 'landsat_9.tif', studyarea)

# Process and save Landsat 8 data
dataset_landsat8 = dataset_landsat8.map(apply_scale_factors)
landsat_8 = dataset_landsat8.median().clip(studyarea)
save_data(out_dir, landsat_8, 'landsat_8.tif', studyarea)


landsat_9.tif: |          | 0.00/4.95M (raw) [  0.0%] in 00:00 (eta:     ?)

landsat_8.tif: |          | 0.00/4.95M (raw) [  0.0%] in 00:00 (eta:     ?)

In [9]:
#Save the LULC data
LULC = get_lulc(studyarea)
geemap.download_ee_image(LULC, '../Data/LULC.tif', scale= 10, region=studyarea, crs='EPSG:4326')



LULC.tif: |          | 0.00/290k (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None
